## Quiz 7 Answers

In [1]:
import pandas as pd                                   # Working with tables.

from tiingo import TiingoClient                       # Stock prices.
import quandl                                         # Economic data, futures prices, ...

# API keys:
tiingo = TiingoClient({'api_key':'XXXX'})
quandl.ApiConfig.api_key = 'YYYY'

In [2]:
PRICE       = tiingo.get_dataframe(['CVX','COST','MSFT','JNJ'],'2000-01-01', metric_name='adjClose')
PRICE.index = pd.to_datetime(PRICE.index).tz_convert(None)
RET = PRICE.pct_change()
PRICE[:3]

,CVX,COST,MSFT,JNJ
2000-01-03,19.364171,31.352158,37.097350,26.893090
2000-01-04,19.364171,29.636596,35.843373,25.907098
2000-01-05,19.783269,30.140344,36.222113,26.309663


Question 4:

In [3]:
(0.5 * RET.CVX.add(1).cumprod() + 0.5 * RET.COST.add(1).cumprod())[:'2020']

2000-01-03         NaN
2000-01-04    0.972640
2000-01-05    0.991496
2000-01-06    1.020948
2000-01-07    1.062747
                ...   
2020-12-24    7.976052
2020-12-28    8.068240
2020-12-29    8.087275
2020-12-30    8.133148
2020-12-31    8.147822
Length: 5284, dtype: float64

Question 5:

In [4]:
(0.5 * RET.CVX + 0.5 * RET.COST).add(1).cumprod()[:'2020']

2000-01-03         NaN
2000-01-04    0.972640
2000-01-05    0.991432
2000-01-06    1.020602
2000-01-07    1.063366
                ...   
2020-12-24    9.579737
2020-12-28    9.640734
2020-12-29    9.645834
2020-12-30    9.709261
2020-12-31    9.689403
Length: 5284, dtype: float64

Question 6:

In [5]:
def get_rebalance_dates(frequency):
    group = getattr(PRICE.index, frequency) 
    return PRICE[:1].index.union(PRICE.groupby([PRICE.index.year, group]).tail(1).index)

frequency = 'quarter'

rebalance_dates = get_rebalance_dates(frequency) 


weights = pd.Series({'CVX':0.3, 'MSFT':0.5, 'JNJ':0.2})

start_date = rebalance_dates[0]
end_date   = rebalance_dates[1]

portfolio_value = pd.Series(1, index=[rebalance_dates[0]])

cum_ret = RET[start_date:end_date][1:].add(1).cumprod()

new_positions = portfolio_value.iloc[-1] * weights 

# Before rebalancing
start_to_end_positions = new_positions  * cum_ret
start_to_end_positions[-3:]

,COST,CVX,JNJ,MSFT
2000-03-29,NaN,0.320122,0.154717,0.459806
2000-03-30,NaN,0.333469,0.154042,0.443420
2000-03-31,NaN,0.334374,0.152954,0.455774


Question 7:

In [6]:
# After rebalancing
start_to_end_positions.sum('columns').iloc[-1] * weights

CVX     0.282930
MSFT    0.471551
JNJ     0.188620
dtype: float64

Question 8:

In [7]:
portfolio_value = portfolio_value.append( start_to_end_positions.sum('columns') ) 

start_date = rebalance_dates[1]
end_date   = rebalance_dates[2]

cum_ret = RET[start_date:end_date][1:].add(1).cumprod()

new_positions = portfolio_value.iloc[-1] * weights   # initial dollar investments

start_to_end_positions = new_positions  * cum_ret

portfolio_value = portfolio_value.append( start_to_end_positions.sum('columns') ) 


start_date = rebalance_dates[2]
end_date   = rebalance_dates[3]

cum_ret = RET[start_date:end_date][1:].add(1).cumprod()

new_positions = portfolio_value.iloc[-1] * weights   # initial dollar investments

start_to_end_positions = new_positions  * cum_ret

portfolio_value = portfolio_value.append( start_to_end_positions.sum('columns') ) 
portfolio_value

2000-01-03    1.000000
2000-01-04    0.975766
2000-01-05    0.990358
2000-01-06    0.991029
2000-01-07    1.011350
                ...   
2000-09-25    0.772260
2000-09-26    0.779545
2000-09-27    0.775873
2000-09-28    0.772531
2000-09-29    0.771469
Length: 189, dtype: float64

In [8]:
# Value end of August:
portfolio_value.loc['2000-8'][-1:]

2000-08-31    0.818583
dtype: float64

Question 10:

In [9]:
def run_backtest(frequency):   
    rebalance_dates = get_rebalance_dates(frequency)
    weights         = pd.Series({'CVX':1/4, 'COST':1/4, 'JNJ':1/4, 'MSFT':1/4})
    portfolio_value = pd.Series(1,                        index=[rebalance_dates[0]])    
    trades          = pd.DataFrame(columns=weights.index, index=[rebalance_dates[0]])
    previous_positions = weights

    for i in range(len(rebalance_dates)-1):
        start_date = rebalance_dates[i]
        end_date   = rebalance_dates[i+1]
                
        cum_ret = RET[start_date:end_date][1:].add(1).cumprod()
        
        new_positions          = portfolio_value.iloc[-1] * weights  # dollars invested at start of this period
        start_to_end_positions = new_positions  * cum_ret
        
        portfolio_value = portfolio_value.append(start_to_end_positions.sum('columns'))  
                           
        trades.loc[start_date] = new_positions - previous_positions
        previous_positions     = start_to_end_positions.iloc[-1]

    return portfolio_value, trades



value, trades = run_backtest('quarter')

turnover = trades.abs().sum('columns').div(2)

turnover.resample('A').sum().div( value.resample('A').mean() )

2000-12-31    0.341677
2001-12-31    0.210833
2002-12-31    0.159811
2003-12-31    0.154611
2004-12-31    0.084633
2005-12-31    0.130947
2006-12-31    0.111373
2007-12-31    0.097075
2008-12-31    0.134147
2009-12-31    0.109480
2010-12-31    0.085626
2011-12-31    0.106011
2012-12-31    0.105958
2013-12-31    0.073154
2014-12-31    0.110738
2015-12-31    0.121993
2016-12-31    0.110745
2017-12-31    0.084451
2018-12-31    0.096967
2019-12-31    0.097872
2020-12-31    0.178785
2021-12-31         NaN
Freq: A-DEC, dtype: float64

&rarr; 8.5% in 2010